In [1]:
import scipy.io
import torch
import importlib
import matplotlib.pyplot as plt
import numpy as np

In [3]:
import src.HyperParameters as hp 
import src.data as dt 
import src.model3 as Model 
import src.train as trainer

importlib.reload(hp)
importlib.reload(dt)
importlib.reload(Model)
importlib.reload(trainer)

<module 'src.train' from 'c:\\Users\\s420553\\Downloads\\F_PINNN\\Friction-Factor-Estimation-PINN\\src\\train.py'>

In [3]:
data = scipy.io.loadmat('/Users/ramtarun/Desktop/Cambridge/Indirect-Noise-in-Nozzles/Data/Data_PINN_subsonic_geom_linvelsup_f0-0.1.mat')

In [4]:
PINN_model = Model.PINN(hp.input_size, hp.output_size, hp.hidden_size, hp.num_layers)
# PINN_model.train()

In [5]:
params = list(PINN_model.parameters())
# optimizer = torch.optim.Adam([{'params' : params[1::]},{'params' : params[-1], 'lr': hp.ff_learning_rate}], lr = hp.learning_rate, amsgrad = True)   
optimizer = torch.optim.Adam(params=params, lr = hp.learning_rate, amsgrad = True)   
# optimizer = torch.optim.LBFGS(params, hp.ff_learning_rate, 
#                               max_iter = hp.epochs, 
#                               max_eval = None, 
#                               tolerance_grad = 1e-11, 
#                               tolerance_change = 1e-11, 
#                               history_size = 100, 
#                               line_search_fn = 'strong_wolfe')


In [6]:
fval = [0.1  , 0.06 , 0.04, 0.01]

# for f in fval:
inputs, targets, meanflow =  dt.DataPreprocessing(data, ff=0.01)
N = inputs.shape[1]
train_loader, val_loader = dt.DataTransformer(inputs, targets, meanflow, TrainingSet=True)
# ### Model 3
# batch_size = 32
# sequence_length = len(inputs) // batch_size
# # Reshape the input tensors within the DataLoader
# train_loader = []
# for batch in Train_loader:
#     input_data = batch.view(batch_size, sequence_length, 1)
#     train_loader.append(input_data)

train_loss, val_loss, f_train, f_test, f_dist = trainer.train(train_loader, val_loader, hp.epochs, optimizer, PINN_model, N)
with torch.no_grad():
    plt.figure() 
    plt.plot(train_loss.keys(), train_loss.values(), 'r-', label='Training Loss')
    plt.plot(val_loss.keys(), val_loss.values(), 'g-', label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()
    plt.show()
    plt.figure()
    plt.plot(f_train.keys(), f_train.values(), 'r-', label='Training Friction Factor')
    plt.plot(f_test.keys(), f_test.values(), 'g-', label='Validation Friction Factor')
    plt.xlabel('Epochs')
    plt.ylabel('Friction Factor')
    plt.legend()
    plt.grid()
    plt.show()
    plt.figure()


Epoch 10/1000 - Train Loss: 0.192116 Val Loss: 0.002611 f_train: 0.0643 f_test: 0.0679
Epoch 20/1000 - Train Loss: 0.179700 Val Loss: 0.002292 f_train: 0.0674 f_test: 0.0776
Epoch 30/1000 - Train Loss: 0.175881 Val Loss: 0.002326 f_train: 0.0782 f_test: 0.0781
Epoch 40/1000 - Train Loss: 0.168891 Val Loss: 0.002599 f_train: 0.0868 f_test: 0.0866
Epoch 50/1000 - Train Loss: 0.170819 Val Loss: 0.002577 f_train: 0.0835 f_test: 0.0850
Epoch 60/1000 - Train Loss: 0.166151 Val Loss: 0.002398 f_train: 0.0978 f_test: 0.0977
Epoch 70/1000 - Train Loss: 0.161913 Val Loss: 0.002385 f_train: 0.0918 f_test: 0.0889
Epoch 80/1000 - Train Loss: 0.163672 Val Loss: 0.002350 f_train: 0.0825 f_test: 0.0815
Epoch 90/1000 - Train Loss: 0.162211 Val Loss: 0.002544 f_train: 0.0935 f_test: 0.0929
Epoch 100/1000 - Train Loss: 0.159608 Val Loss: 0.002264 f_train: 0.0955 f_test: 0.0961
Epoch 110/1000 - Train Loss: 0.159599 Val Loss: 0.002099 f_train: 0.1003 f_test: 0.0952
Epoch 120/1000 - Train Loss: 0.158260 Val

: 

: 

In [ ]:
plt.figure()
plt.plot(inputs[:,1], targets)
preds = PINN_model.rnn(inputs).detach()
plt.plot(inputs[:,1], preds[:,:3], 'k-.')
